In [ ]:
# Today I found a framework for process-based discrete-event simulation 
# and tried to execite an example 
# (soure code from https://simpy.readthedocs.io/en/latest/examples/gas_station_refuel.html)


In [7]:
# Python 3.7.*, simpy 3.0.11

"""
Gas Station Refueling example

Covers:

- Resources: Resource
- Resources: Container
- Waiting for other processes

Scenario:
  A gas station has a limited number of gas pumps that share a common
  fuel reservoir. Cars randomly arrive at the gas station, request one
  of the fuel pumps and start refueling from that reservoir.

  A gas station control process observes the gas station's fuel level
  and calls a tank truck for refueling if the station's level drops
  below a threshold.

"""

import itertools
import random

import simpy


RANDOM_SEED = 42
GAS_STATION_SIZE = 200     # Liters
THRESHOLD = 20             # Threshold for calling the tank truck (in %)
FUEL_TANK_SIZE = 50        # Liters
FUEL_TANK_LEVEL = [5, 25]  # Min/max levels of fuel tanks (in liters)
REFUELING_SPEED = 2        # liters / second
TANK_TRUCK_TIME = 300      # Seconds it takes the tank truck to arrive
T_INTER = [30, 300]        # Create a car every [min, max] seconds
SIM_TIME = 100000           # Simulation time in seconds


def car(name: str,
        env_: simpy.Environment,
        gas_station_: simpy.Resource,
        fuel_pump_: simpy.Container) -> None:
    """A car arrives at the gas station for refueling.

    It requests one of the gas station's fuel pumps and tries to get the
    desired amount of gas from it. If the stations reservoir is
    depleted, the car has to wait for the tank truck to arrive.

    """
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print(f'{name} arriving at gas station at {env_.now}')
    with gas_station_.request() as req:
        start = env_.now
        # Request one of the gas pumps
        yield req

        # Get the required amount of fuel
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        yield fuel_pump_.get(liters_required)

        # The "actual" refueling process takes some time
        yield env_.timeout(liters_required / REFUELING_SPEED)

        print(f'{name} finished refueling in {env_.now - start} seconds.')


def gas_station_control(env_: simpy.Environment,
                        fuel_pump_: simpy.Container) -> None:
    """Periodically check the level of the *fuel_pump* and call the tank
    truck if the level falls below a threshold."""

    while True:
        if fuel_pump_.level / fuel_pump_.capacity * 100 < THRESHOLD:
            # We need to call the tank truck now!
            print(f'Calling tank truck at {env_.now}')
            # Wait for the tank truck to arrive and refuel the station
            yield env_.process(tank_truck(env_, fuel_pump_))

        yield env.timeout(10)  # Check every 10 seconds


def tank_truck(env_: simpy.Environment,
               fuel_pump_: simpy.Container) -> None:
    """Arrives at the gas station after a certain delay and refuels it."""

    yield env_.timeout(TANK_TRUCK_TIME)
    print(f'Tank truck arriving at time {env_.now}')
    amount = fuel_pump_.capacity - fuel_pump_.level
    print(f'Tank truck refuelling {amount} liters.')
    yield fuel_pump_.put(amount)


def car_generator(env_: simpy.Environment,
                  gas_station_: simpy.Resource,
                  fuel_pump_: simpy.Container) -> None:
    """Generate new cars that arrive at the gas station."""

    for i in itertools.count():
        yield env_.timeout(random.randint(*T_INTER))
        env_.process(car(f'Car {i}', env_, gas_station_, fuel_pump_))


# Setup and start the simulation
print('Gas Station refuelling')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()
gas_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, GAS_STATION_SIZE, init=GAS_STATION_SIZE)
env.process(gas_station_control(env, fuel_pump))
env.process(car_generator(env, gas_station, fuel_pump))

# Execute!
env.run(until=SIM_TIME)


Gas Station refuelling
Car 0 arriving at gas station at 87
Car 0 finished refueling in 18.5 seconds.
Car 1 arriving at gas station at 129
Car 1 finished refueling in 19.0 seconds.
Car 2 arriving at gas station at 284
Car 2 finished refueling in 21.0 seconds.
Car 3 arriving at gas station at 385
Car 3 finished refueling in 13.5 seconds.
Car 4 arriving at gas station at 459
Calling tank truck at 460
Car 4 finished refueling in 22.0 seconds.
Car 5 arriving at gas station at 705
Car 6 arriving at gas station at 750
Tank truck arriving at time 760
Tank truck refuelling 188 liters.
Car 6 finished refueling in 29.0 seconds.
Car 5 finished refueling in 76.5 seconds.
Car 7 arriving at gas station at 891
Car 7 finished refueling in 13.0 seconds.
Car 8 arriving at gas station at 1179
Car 8 finished refueling in 14.0 seconds.
Car 9 arriving at gas station at 1222
Car 9 finished refueling in 12.5 seconds.
Car 10 arriving at gas station at 1353
Calling tank truck at 1360
Car 10 finished refueling in